# 從Wikipedia中爬取文章。先定義一個搜尋的關鍵字，擷取該關鍵字詞的文章

# 定義一個爬蟲函數，這個函數的主要工作為：
 # (1) 爬取當前關鍵字的解釋，並存入檔案(因為文章內容太多會佔滿整個頁面，所以存程檔案，方便後續檢視)

 # (2) 萃取出當前關鍵字所引用的外部連結，當作新的查詢關鍵字

 #  (3) 把第(2)擷取到的關鍵字當作新的關鍵字，回到第(1)步，爬取新的關鍵字解釋

In [28]:
import requests
from bs4 import BeautifulSoup
import re   #將網站轉換成 編碼後的url
import os
from urllib.request import urlretrieve 



In [40]:
def WikiArticle(key_word_link, key_word, recursive):
     if (recursive <= max_recursive_depth):
        print("遞迴層[%d] -%s (%s)" % (recursive, key_word_link, key_word)) 
        # 模擬封包的標頭
        headers = {
            'authority': 'zh.wikipedia.org',
            'method': 'GET',
            'path': '/wiki/' + key_word_link,
            'scheme': 'https',
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
            'accept-encoding': 'gzip, deflate, br',
            'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
            'cookie': 'GeoIP=TW:TPE:Taipei:25.05:121.53:v4; TBLkisOn=0; mwPhp7Seed=8b8; WMF-Last-Access-Global=04-Jun-2019; WMF-Last-Access=04-Jun-2019',
            'dnt': '1',
            'if-modified-since': 'Tue, 04 Jun 2019 12:03:22 GMT',
            'referer': 'https://zh.wikipedia.org/wiki/Wikipedia:%E9%A6%96%E9%A1%B5',
            'upgrade-insecure-requests': '1',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
        }    
        '''
        ---------------以下為查詢關鍵字與擷取,儲存內容------------------
        '''
        url = 'https://zh.wikipedia.org' + key_word_link  # 組合關鍵字查詢URL
        resp = requests.get(url, headers=headers)
        resp.encoding = 'utf-8'

        html = BeautifulSoup(resp.text, "lxml")
        content = html.find(name='div', attrs={'id':'mw-content-text'}).find_all(name='p')
        #
        # Part 1: 爬取當前關鍵字的文章內容。
        #         因為內容太多，我們把它寫入檔案，並以關鍵字作為檔案名稱，以便稍後查閱內容。
        #         請先建立一個名為"WikiArticle"的資料夾，爬取到的文章內容會放在這個資料夾底下。
        #
   
        if not os.path.exists('./WikiArticle/'):
            os.makedirs( './WikiArticle/')  
        key_word=key_word.replace("/", "_");

        f = open('./WikiArticle/'+key_word+'.txt', "w",encoding="utf-8")    
        for string in content:
                file_name = key_word.replace('/', '_')  # 以'_'取代'/', 避免檔名被誤解釋成路徑
                f = open("WikiArticle/" + file_name, "a+", encoding='utf-8')
                f.write(string.text)
        f.close()
        '''
        --------------儲存成dict 以便使用 recursive -----------
        1.特別注意 key_word=key_word.replace("/", "_");
        檔名的設定要特別注意
        不能包括
        \/>?  ..  etc.
        '''
        #
        # Part 2: 萃取出本篇文章中所延伸引用的外部連結，並儲存在external_link_dict
        #
        external_link_dict = dict({})
        for i in content:
            a_tag=i.find_all( ("a") , href=re.compile("^/wiki/"))
            if len(a_tag) > 0:
                for ex_url in a_tag:
                        ex_link=ex_url['href']    # 外部連結的網址
                        ex_title=ex_url.get_text()  # 外部連結的中文名稱
                        external_link_dict[ ex_link ] = ex_title
              #  print(ex_title , ":","https://zh.wikipedia.org"+ex_link)
#        data.close()
       # Part 3: 將Part 2所收集的外部連結，當作新的關鍵字，繼續迭代深入爬蟲
        #
        if (len(external_link_dict) > 0):
            
            recursive = recursive + 1  # 遞迴深度加1
            
            for k, v in external_link_dict.items():   # 只是把  "" : " "拿掉((  K , v  只是隨便的變數 這邊用上了ex_link_dict中的數據
                WikiArticle(k, v, recursive)  # 再次呼叫同樣的函數，執行同樣的流程 '''
                

# 定義一個我們想搜尋的字詞，並將它轉換成UTF-8編碼後的URL

In [41]:
input_keyword="網路爬蟲"  # define the word we wanna search

url_utf8=repr(input_keyword.encode("UTF-8")).upper()  #representation # 編碼成UTF-8並轉成大寫字元
url_utf8 = url_utf8.replace("\\X", "%") #replace \x to %
print(url_utf8)
print("%s: %s" % (input_keyword, url_utf8[2:-1:1]))     # 擷取中間的編碼結果

# 組成Wiki關鍵字搜尋的網址格式
root_keyword_link = '/wiki/' + url_utf8[2:-1:1]   # 家一下
print(root_keyword_link)

B'%E7%B6%B2%E8%B7%AF%E7%88%AC%E8%9F%B2'
網路爬蟲: %E7%B6%B2%E8%B7%AF%E7%88%AC%E8%9F%B2
/wiki/%E7%B6%B2%E8%B7%AF%E7%88%AC%E8%9F%B2


# 定義爬取的遞迴深度。深度不要訂太深，否則會爬很久。

In [42]:
max_recursive_depth = 2

WikiArticle(root_keyword_link, input_keyword, 0)

遞迴層[0] -/wiki/%E7%B6%B2%E8%B7%AF%E7%88%AC%E8%9F%B2 (網路爬蟲)
遞迴層[1] -/wiki/%E4%B8%87%E7%BB%B4%E7%BD%91 (全球資訊網)
遞迴層[2] -/wiki/%E4%BA%92%E8%81%94%E7%BD%91 (網際網路)
遞迴層[2] -/wiki/%E8%B6%85%E6%96%87%E6%9C%AC (超文字)
遞迴層[2] -/wiki/%E8%92%82%E5%A7%86%C2%B7%E4%BC%AF%E7%BA%B3%E6%96%AF-%E6%9D%8E (提姆·柏內茲-李)
遞迴層[2] -/wiki/CERN (CERN)
遞迴層[2] -/wiki/%E7%B6%B2%E9%A0%81%E7%80%8F%E8%A6%BD%E5%99%A8 (網頁瀏覽器)
遞迴層[2] -/wiki/%E8%B3%87%E8%A8%8A%E6%99%82%E4%BB%A3 (資訊時代)
遞迴層[2] -/wiki/%E7%B6%B2%E9%A0%81 (網頁)
遞迴層[2] -/wiki/%E6%A0%BC%E5%BC%8F%E5%8C%96%E6%96%87%E6%9C%AC (格式化)
遞迴層[2] -/wiki/%E8%B6%85%E6%96%87%E4%BB%B6%E6%A8%99%E7%A4%BA%E8%AA%9E%E8%A8%80 (超文件標示語言)
遞迴層[2] -/wiki/%E5%9C%96%E7%89%87 (圖片)
遞迴層[2] -/wiki/%E5%BD%B1%E7%89%87 (影片)
遞迴層[2] -/wiki/%E8%81%B2%E9%9F%B3 (聲音)
遞迴層[2] -/wiki/%E5%A4%9A%E5%AA%92%E9%AB%94 (多媒體)
遞迴層[2] -/wiki/%E7%B6%B2%E9%9A%9B%E7%B6%B2%E8%B7%AF (網際網路)
遞迴層[2] -/wiki/ENQUIRE (ENQUIRE)
遞迴層[2] -/wiki/%E6%95%B0%E6%8D%AE%E5%BA%93 (資料庫)
遞迴層[2] -/wiki/%E8%AF%AD%E4%B9%89%E7%BD%91 (語意網)
遞迴層[2] -/wiki/%E

遞迴層[2] -/wiki/Google (Google)
遞迴層[2] -/wiki/Yahoo (Yahoo)
遞迴層[2] -/wiki/Bing (Bing)
遞迴層[2] -/wiki/Ask.com (Ask)
遞迴層[2] -/wiki/%E4%BA%92%E8%81%94%E7%BD%91%E6%A1%A3%E6%A1%88%E9%A6%86 (網際網路檔案館)
遞迴層[1] -/wiki/%E7%BD%91%E7%AB%99 (網站)
遞迴層[2] -/wiki/%E8%90%AC%E7%B6%AD%E7%B6%B2 (全球資訊網)
遞迴層[2] -/wiki/HTML (HTML)
遞迴層[2] -/wiki/%E7%B6%B2%E9%A0%81 (網頁)
遞迴層[2] -/wiki/%E7%BD%91%E7%BB%9C%E6%9C%8D%E5%8A%A1 (網路服務)
遞迴層[2] -/wiki/%E7%B6%B2%E9%A0%81%E7%80%8F%E8%A6%BD%E5%99%A8 (網頁瀏覽器)
遞迴層[2] -/wiki/%E6%8B%9B%E8%81%98 (招聘)
遞迴層[2] -/wiki/%E5%80%8B%E4%BA%BA%E4%B8%BB%E9%A0%81 (個人主頁)
遞迴層[2] -/wiki/%E6%96%B0%E8%81%9E (新聞)
遞迴層[2] -/wiki/%E6%97%85%E9%81%8A (旅遊)
遞迴層[2] -/wiki/%E5%A8%9B%E6%A8%82 (娛樂)
遞迴層[2] -/wiki/%E7%B6%93%E6%BF%9F (經濟)
遞迴層[2] -/wiki/%E5%9C%96%E5%83%8F (圖像)
遞迴層[2] -/wiki/%E8%81%B2%E9%9F%B3 (聲音)
遞迴層[2] -/wiki/%E5%8B%95%E7%95%AB (動畫)
遞迴層[2] -/wiki/%E5%BD%B1%E7%89%87 (影片)
遞迴層[2] -/wiki/3D (3D)
遞迴層[2] -/wiki/%E5%8B%95%E6%85%8B%E7%B6%B2%E9%A0%81 (動態網頁)
遞迴層[2] -/wiki/%E9%9B%BB%E5%AD%90%E9%83%B5%E4%BB%B6 

遞迴層[2] -/wiki/%E7%B6%B2%E7%AB%99 (網站)
遞迴層[2] -/wiki/%E9%A6%96%E9%A0%81 (首頁)
遞迴層[2] -/wiki/%E6%96%87%E5%AD%97%E7%B7%A8%E8%BC%AF%E5%99%A8 (文字編輯器)
遞迴層[2] -/wiki/HTML%E7%B7%A8%E8%BC%AF%E5%99%A8 (HTML編輯器)
遞迴層[2] -/wiki/FTP (FTP)
遞迴層[2] -/wiki/%E4%B8%8A%E8%BC%89 (上傳)
遞迴層[1] -/wiki/%E4%BA%92%E8%81%94%E7%BD%91 (網際網路)
遞迴層[2] -/wiki/20%E4%B8%96%E7%B4%80 (20世紀)
遞迴層[2] -/wiki/%E7%94%B5%E8%84%91%E7%BD%91%E7%BB%9C (電腦網路)
遞迴層[2] -/wiki/%E7%B6%B2%E8%B7%AF (網路)
遞迴層[2] -/wiki/%E7%BD%91%E7%BB%9C%E5%8D%8F%E8%AE%AE (網路協定)
遞迴層[2] -/wiki/%E5%AD%B8%E8%A1%93%E7%95%8C (學術界)
遞迴層[2] -/wiki/%E7%94%B5%E5%AD%90%E6%8A%80%E6%9C%AF (電子)
遞迴層[2] -/wiki/%E7%84%A1%E7%B7%9A (無線)
遞迴層[2] -/wiki/%E5%85%89%E7%BA%96%E7%B6%B2%E8%B7%AF (光纖網路技術)
遞迴層[2] -/wiki/%E8%B6%85%E6%96%87%E6%9C%AC (超文字)
遞迴層[2] -/wiki/%E8%90%AC%E7%B6%AD%E7%B6%B2 (全球資訊網)
遞迴層[2] -/wiki/%E9%9B%BB%E5%AD%90%E9%83%B5%E4%BB%B6 (電子郵件)
遞迴層[2] -/wiki/%E7%B6%B2%E9%9A%9B%E5%8D%94%E8%AD%B0%E9%80%9A%E8%A9%B1%E6%8A%80%E8%A1%93 (通話)
遞迴層[2] -/wiki/%E6%AA%94%E6%A1%88%E5%88%86%E

遞迴層[2] -/wiki/%E4%B8%AD%E5%A4%AE%E5%A4%84%E7%90%86%E5%99%A8 (中央處理器)
遞迴層[2] -/wiki/%E7%A1%AC%E7%9B%98 (硬碟)
遞迴層[2] -/wiki/%E5%86%85%E5%AD%98 (記憶體)
遞迴層[2] -/wiki/%E7%A1%AC%E4%BB%B6 (硬體)
遞迴層[2] -/wiki/PC (PC)
遞迴層[2] -/wiki/%E7%B5%82%E7%AB%AF (終端)
遞迴層[2] -/wiki/%E5%AE%A2%E6%88%B7%E7%AB%AF (客戶端)
遞迴層[2] -/wiki/%E5%80%8B%E4%BA%BA%E9%9B%BB%E8%85%A6 (個人電腦)
遞迴層[2] -/wiki/%E5%8F%AF%E9%9D%A0%E6%80%A7%E3%80%81%E5%8F%AF%E7%94%A8%E6%80%A7%E5%92%8C%E5%8F%AF%E7%B6%AD%E8%AD%B7%E6%80%A7 (RAS)
遞迴層[2] -/wiki/%E7%BA%A0%E9%94%99%E5%86%85%E5%AD%98 (ECC記憶體)
遞迴層[2] -/wiki/%E6%93%8D%E4%BD%9C%E7%B3%BB%E7%BB%9F (作業系統)
遞迴層[2] -/wiki/%E7%B1%BBUnix (類Unix)
遞迴層[2] -/wiki/Unix (Unix)
遞迴層[2] -/wiki/Linux%E5%8F%91%E8%A1%8C%E7%89%88 (Linux發行版)
遞迴層[2] -/wiki/Red_Hat_Enterprise_Linux (紅帽企業Linux)
遞迴層[2] -/wiki/SUSE (SUSE)
遞迴層[2] -/wiki/AIX (AIX)
遞迴層[2] -/wiki/IRIX (IRIX)
遞迴層[2] -/wiki/Solaris (Solaris)
遞迴層[2] -/wiki/BSD (BSD)
遞迴層[2] -/wiki/OpenBSD (OpenBSD)
遞迴層[2] -/wiki/NetBSD (NetBSD)
遞迴層[2] -/wiki/%E5%BE%AE%E8%BD%AF (微軟)
遞